In [1]:
#import libraries
import glob
from keras.models import Sequential, load_model
import numpy as np
import pandas as pd
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt
import keras as k
import datetime
from datetime import datetime, timedelta

In [2]:
# Importing the dataset
dataset = pd.read_csv('./RawData/current_data.csv')

dataset.head()


/var/folders/cl/wxhg84_d06q47nm4f6nr73500000gn/T/ipykernel_69928/3421204070.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('./RawData/current_data.csv')


,_id,mac,CT1,CT2,CT3,CT_Avg,total_current,state,state_duration,fault_status,fw_version,machineId,created_at,__v,mqtt_time,date_now
0,66015cf84c41115da7019c0b,08:D1:F9:A5:C8:84,0.0,0.0,0.0,0.0,0.0,OFF,31,NOT_FAULTY,1.5,66015cd44c41115da7019b0f,2024-03-25 11:16:07,0,NaN,NaN
1,66015cff4c41115da7019c35,08:D1:F9:A5:C8:84,0.0,0.0,0.0,0.0,0.0,OFF,38,NOT_FAULTY,1.5,66015cd44c41115da7019b0f,2024-03-25 11:16:14,0,NaN,NaN
2,66015d064c41115da7019c64,08:D1:F9:A5:C8:84,0.0,0.0,0.0,0.0,0.0,OFF,45,NOT_FAULTY,1.5,66015cd44c41115da7019b0f,2024-03-25 11:16:21,0,NaN,NaN
3,66015d0d4c41115da7019c8a,08:D1:F9:A5:C8:84,0.0,0.0,0.0,0.0,0.0,OFF,52,NOT_FAULTY,1.5,66015cd44c41115da7019b0f,2024-03-25 11:16:28,0,NaN,NaN
4,66015d144c41115da7019cb7,08:D1:F9:A5:C8:84,0.0,0.0,0.0,0.0,0.0,OFF,59,NOT_FAULTY,1.5,66015cd44c41115da7019b0f,2024-03-25 11:16:35,0,NaN,NaN


In [3]:
dataset.shape

(1276069, 16)

In [4]:
#Creating a list of column names to keep
columns_to_retain = ['CT1', 'CT2', 'CT3', 'CT_Avg', 'total_current', 'state', 'created_at']

#Drop the columns that are not wanted in columns_to_retain
dataset = dataset[columns_to_retain]

pd.DataFrame(dataset)

,CT1,CT2,CT3,CT_Avg,total_current,state,created_at
0,0.0,0.000000,0.0,0.000000,0.000000,OFF,2024-03-25 11:16:07
1,0.0,0.000000,0.0,0.000000,0.000000,OFF,2024-03-25 11:16:14
2,0.0,0.000000,0.0,0.000000,0.000000,OFF,2024-03-25 11:16:21
3,0.0,0.000000,0.0,0.000000,0.000000,OFF,2024-03-25 11:16:28
4,0.0,0.000000,0.0,0.000000,0.000000,OFF,2024-03-25 11:16:35
...,...,...,...,...,...,...,...
1276064,0.0,0.000000,0.0,0.000000,0.000000,OFF,2024-08-11 21:55:50
1276065,0.0,0.125287,0.0,0.041762,0.125287,OFF,2024-08-11 21:55:55
1276066,0.0,0.000000,0.0,0.000000,0.000000,OFF,2024-08-11 21:56:00
1276067,0.0,0.000000,0.0,0.000000,0.000000,OFF,2024-08-11 21:56:05


In [5]:
# Drop rows where state is not ON
dataset = dataset[dataset.state != 'OFF']

# drop idle
dataset = dataset[dataset.state != 'IDLE']

pd.DataFrame(dataset)

,CT1,CT2,CT3,CT_Avg,total_current,state,created_at
2878,0.000000,11.271318,14.648021,8.639779,25.919338,ON,2024-03-26 04:17:37
4068,12.206779,13.903328,10.293859,12.134656,36.403969,ON,2024-03-26 06:07:21
17696,1.157038,1.262079,1.147739,1.188952,3.566856,ON,2024-03-27 01:17:40
17697,1.429621,1.462852,1.459629,1.450701,4.352102,ON,2024-03-27 01:17:45
17698,1.383395,1.581238,1.362355,1.442329,4.326988,ON,2024-03-27 01:17:50
...,...,...,...,...,...,...,...
1248270,1.190704,1.083099,1.135036,1.136280,3.408839,ON,2024-08-10 06:18:13
1248271,3.768695,3.919246,3.917646,3.868529,11.605587,ON,2024-08-10 06:18:18
1248272,3.933802,3.982554,3.588322,3.834893,11.504679,ON,2024-08-10 06:18:23
1248278,1.279599,1.272777,1.131611,1.227996,3.683987,ON,2024-08-10 06:18:53


In [6]:
# Convert created_at 2024-03-26 04:17:37 format to 2024-08-11T20:29:06.000Z format

# Convert to datetime object ie datetime.strptime


dataset['created_at'] = pd.to_datetime(dataset['created_at'])

pd.to_timedelta(dataset.groupby('created_at').cumcount(), unit='ms')

dataset['created_at'] = dataset['created_at'] + pd.to_timedelta(dataset.groupby('created_at').cumcount(), unit='ms')


dataset['created_at'] = dataset['created_at'].dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')



In [7]:

pd.DataFrame(dataset)

,CT1,CT2,CT3,CT_Avg,total_current,state,created_at
2878,0.000000,11.271318,14.648021,8.639779,25.919338,ON,2024-03-26T04:17:37.000000Z
4068,12.206779,13.903328,10.293859,12.134656,36.403969,ON,2024-03-26T06:07:21.000000Z
17696,1.157038,1.262079,1.147739,1.188952,3.566856,ON,2024-03-27T01:17:40.000000Z
17697,1.429621,1.462852,1.459629,1.450701,4.352102,ON,2024-03-27T01:17:45.000000Z
17698,1.383395,1.581238,1.362355,1.442329,4.326988,ON,2024-03-27T01:17:50.000000Z
...,...,...,...,...,...,...,...
1248270,1.190704,1.083099,1.135036,1.136280,3.408839,ON,2024-08-10T06:18:13.000000Z
1248271,3.768695,3.919246,3.917646,3.868529,11.605587,ON,2024-08-10T06:18:18.000000Z
1248272,3.933802,3.982554,3.588322,3.834893,11.504679,ON,2024-08-10T06:18:23.000000Z
1248278,1.279599,1.272777,1.131611,1.227996,3.683987,ON,2024-08-10T06:18:53.000000Z


In [8]:
# remove cleandata folder and make new one

import os

os.system('rm -rf CleanData')
os.system('mkdir CleanData')




0

In [9]:
dataset['created_at'].value_counts()

created_at
2024-03-26T04:17:37.000000Z    1
2024-07-21T05:06:29.000000Z    1
2024-07-21T05:18:30.000000Z    1
2024-07-21T05:18:35.000000Z    1
2024-07-21T05:18:50.000000Z    1
                              ..
2024-05-23T03:12:42.000000Z    1
2024-05-23T03:12:47.000000Z    1
2024-05-23T03:12:52.000000Z    1
2024-05-23T03:12:57.000000Z    1
2024-08-10T06:19:13.000000Z    1
Name: count, Length: 15446, dtype: int64

In [10]:
# Saving the dataset to a new csv file

dataset.to_csv('./CleanData/current_data.csv', index=False)

In [11]:
# Load the new dataset
dataset = pd.read_csv('./RawData/vibration_data.csv')

dataset.head()

,_id,machineId,mac,vibration,created_at,__v
0,66015e6b4c41115da701a564,66015cd44c41115da7019b0f,08:D1:F9:A5:C8:84,1875,2024-03-25 11:22:16,0
1,66015e6f4c41115da701a58a,66015cd44c41115da7019b0f,08:D1:F9:A5:C8:84,1890,2024-03-25 11:22:20,0
2,66015e744c41115da701a5ae,66015cd44c41115da7019b0f,08:D1:F9:A5:C8:84,1872,2024-03-25 11:22:25,0
3,66015e794c41115da701a5d4,66015cd44c41115da7019b0f,08:D1:F9:A5:C8:84,1888,2024-03-25 11:22:30,0
4,66015e7e4c41115da701a5f8,66015cd44c41115da7019b0f,08:D1:F9:A5:C8:84,1895,2024-03-25 11:22:35,0


In [12]:
#Creating a list of column names to keep
columns_to_retain = ['vibration', 'created_at']

#Drop the columns that are not wanted in columns_to_retain
dataset = dataset[columns_to_retain]

pd.DataFrame(dataset)

,vibration,created_at
0,1875,2024-03-25 11:22:16
1,1890,2024-03-25 11:22:20
2,1872,2024-03-25 11:22:25
3,1888,2024-03-25 11:22:30
4,1895,2024-03-25 11:22:35
...,...,...
1284687,1850,2024-08-11 21:56:10
1284688,1888,2024-08-11 21:56:15
1284689,1873,2024-08-11 21:56:20
1284690,1877,2024-08-11 21:56:25


In [13]:
# Drop rows where vibration is zero or missing
dataset = dataset[dataset.vibration != 0]

# dropna function is used to remove missing values
dataset = dataset.dropna(axis=0)

pd.DataFrame(dataset)

,vibration,created_at
0,1875,2024-03-25 11:22:16
1,1890,2024-03-25 11:22:20
2,1872,2024-03-25 11:22:25
3,1888,2024-03-25 11:22:30
4,1895,2024-03-25 11:22:35
...,...,...
1284687,1850,2024-08-11 21:56:10
1284688,1888,2024-08-11 21:56:15
1284689,1873,2024-08-11 21:56:20
1284690,1877,2024-08-11 21:56:25


In [14]:
# Convert created_at 2024-03-26 04:17:37 format to 2024-08-11T20:29:06.000Z format

# Convert to datetime object ie datetime.strptime


dataset['created_at'] = pd.to_datetime(dataset['created_at'])

pd.to_timedelta(dataset.groupby('created_at').cumcount(), unit='ms')

dataset['created_at'] = dataset['created_at'] + pd.to_timedelta(dataset.groupby('created_at').cumcount(), unit='ms')


dataset['created_at'] = dataset['created_at'].dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')



In [15]:
dataset['created_at'].value_counts()

created_at
2024-03-25T11:22:16.000000Z    1
2024-07-14T11:12:39.000000Z    1
2024-07-14T11:13:20.000000Z    1
2024-07-14T11:13:15.000000Z    1
2024-07-14T11:13:10.000000Z    1
                              ..
2024-05-12T20:20:31.000000Z    1
2024-05-12T20:20:26.000000Z    1
2024-05-12T20:20:21.000000Z    1
2024-05-12T20:20:16.000000Z    1
2024-08-11T21:56:30.000000Z    1
Name: count, Length: 1284691, dtype: int64

In [16]:
# Saving the dataset to a new csv file
dataset.to_csv('./CleanData/vibration_data.csv', index=False)

In [17]:
# loading data from the csv files
dataset = pd.read_csv('./RawData/temperature_data.csv')

dataset.head()

,_id,machineId,mac,therm_temp,created_at,__v
0,66015e6b4c41115da701a563,66015cd44c41115da7019b0f,08:D1:F9:A5:C8:84,33.991894,2024-03-25 11:22:16,0
1,66015e6f4c41115da701a588,66015cd44c41115da7019b0f,08:D1:F9:A5:C8:84,33.997105,2024-03-25 11:22:20,0
2,66015e744c41115da701a5ad,66015cd44c41115da7019b0f,08:D1:F9:A5:C8:84,33.977913,2024-03-25 11:22:25,0
3,66015e794c41115da701a5d2,66015cd44c41115da7019b0f,08:D1:F9:A5:C8:84,34.017391,2024-03-25 11:22:30,0
4,66015e7e4c41115da701a5f7,66015cd44c41115da7019b0f,08:D1:F9:A5:C8:84,33.976311,2024-03-25 11:22:35,0


In [18]:
#Creating a list of column names to keep
columns_to_retain = ['therm_temp', 'created_at']

#Drop the columns that are not wanted in columns_to_retain
dataset = dataset[columns_to_retain]

pd.DataFrame(dataset)


,therm_temp,created_at
0,33.991894,2024-03-25 11:22:16
1,33.997105,2024-03-25 11:22:20
2,33.977913,2024-03-25 11:22:25
3,34.017391,2024-03-25 11:22:30
4,33.976311,2024-03-25 11:22:35
...,...,...
1284380,29.894962,2024-08-11 21:58:10
1284381,29.911869,2024-08-11 21:58:15
1284382,29.917768,2024-08-11 21:58:20
1284383,29.895063,2024-08-11 21:58:25


In [19]:
# Drop rows where vibration is zero or missing
dataset = dataset[dataset.therm_temp != 0]

# dropna function is used to remove missing values
dataset = dataset.dropna(axis=0)

pd.DataFrame(dataset)

,therm_temp,created_at
0,33.991894,2024-03-25 11:22:16
1,33.997105,2024-03-25 11:22:20
2,33.977913,2024-03-25 11:22:25
3,34.017391,2024-03-25 11:22:30
4,33.976311,2024-03-25 11:22:35
...,...,...
1284380,29.894962,2024-08-11 21:58:10
1284381,29.911869,2024-08-11 21:58:15
1284382,29.917768,2024-08-11 21:58:20
1284383,29.895063,2024-08-11 21:58:25


In [20]:
# Convert created_at 2024-03-26 04:17:37 format to 2024-08-11T20:29:06.000Z format

# Convert to datetime object ie datetime.strptime


dataset['created_at'] = pd.to_datetime(dataset['created_at'])

pd.to_timedelta(dataset.groupby('created_at').cumcount(), unit='ms')

dataset['created_at'] = dataset['created_at'] + pd.to_timedelta(dataset.groupby('created_at').cumcount(), unit='ms')


dataset['created_at'] = dataset['created_at'].dt.strftime('%Y-%m-%dT%H:%M:%S.%fZ')



In [21]:
dataset['created_at'].value_counts()

created_at
2024-03-25T11:22:16.000000Z    1
2024-07-14T11:22:50.000000Z    1
2024-07-14T11:23:25.000000Z    1
2024-07-14T11:23:20.000000Z    1
2024-07-14T11:23:15.000000Z    1
                              ..
2024-05-12T20:27:17.000000Z    1
2024-05-12T20:27:12.000000Z    1
2024-05-12T20:27:07.000000Z    1
2024-05-12T20:27:02.000000Z    1
2024-08-11T21:58:30.000000Z    1
Name: count, Length: 1284382, dtype: int64

In [22]:
# Saving the dataset to a new csv file
dataset.to_csv('./CleanData/temperature_data.csv', index=False)

In [23]:
# loading cleaned datasets and merging them together interms of time
cts = pd.read_csv('./CleanData/current_data.csv')
temps = pd.read_csv('./CleanData/temperature_data.csv')
vibrations = pd.read_csv('./CleanData/vibration_data.csv')

# Merge the datasets
data = pd.merge(cts, temps, on='created_at')

data = pd.merge(data, vibrations, on='created_at')

pd.DataFrame(data)

,CT1,CT2,CT3,CT_Avg,total_current,state,created_at,therm_temp,vibration
0,0.000000,11.271318,14.648021,8.639779,25.919338,ON,2024-03-26T04:17:37.000000Z,26.383165,1902
1,12.206779,13.903328,10.293859,12.134656,36.403969,ON,2024-03-26T06:07:21.000000Z,31.782297,1903
2,1.157038,1.262079,1.147739,1.188952,3.566856,ON,2024-03-27T01:17:40.000000Z,25.015078,1850
3,1.429621,1.462852,1.459629,1.450701,4.352102,ON,2024-03-27T01:17:45.000000Z,24.980755,1931
4,1.383395,1.581238,1.362355,1.442329,4.326988,ON,2024-03-27T01:17:50.000000Z,25.004904,1815
...,...,...,...,...,...,...,...,...,...
15404,1.190704,1.083099,1.135036,1.136280,3.408839,ON,2024-08-10T06:18:13.000000Z,41.699657,1873
15405,3.768695,3.919246,3.917646,3.868529,11.605587,ON,2024-08-10T06:18:18.000000Z,41.670242,1901
15406,3.933802,3.982554,3.588322,3.834893,11.504679,ON,2024-08-10T06:18:23.000000Z,41.643696,1823
15407,1.279599,1.272777,1.131611,1.227996,3.683987,ON,2024-08-10T06:18:53.000000Z,41.705498,1769


In [24]:
# Dropping the rows with state as 'IDLE'
data = data[data.state != 'IDLE']

pd.DataFrame(data)

,CT1,CT2,CT3,CT_Avg,total_current,state,created_at,therm_temp,vibration
0,0.000000,11.271318,14.648021,8.639779,25.919338,ON,2024-03-26T04:17:37.000000Z,26.383165,1902
1,12.206779,13.903328,10.293859,12.134656,36.403969,ON,2024-03-26T06:07:21.000000Z,31.782297,1903
2,1.157038,1.262079,1.147739,1.188952,3.566856,ON,2024-03-27T01:17:40.000000Z,25.015078,1850
3,1.429621,1.462852,1.459629,1.450701,4.352102,ON,2024-03-27T01:17:45.000000Z,24.980755,1931
4,1.383395,1.581238,1.362355,1.442329,4.326988,ON,2024-03-27T01:17:50.000000Z,25.004904,1815
...,...,...,...,...,...,...,...,...,...
15404,1.190704,1.083099,1.135036,1.136280,3.408839,ON,2024-08-10T06:18:13.000000Z,41.699657,1873
15405,3.768695,3.919246,3.917646,3.868529,11.605587,ON,2024-08-10T06:18:18.000000Z,41.670242,1901
15406,3.933802,3.982554,3.588322,3.834893,11.504679,ON,2024-08-10T06:18:23.000000Z,41.643696,1823
15407,1.279599,1.272777,1.131611,1.227996,3.683987,ON,2024-08-10T06:18:53.000000Z,41.705498,1769


In [25]:
# Dropona function is used to remove missing values

data = data.dropna(axis=0)

pd.DataFrame(data)

,CT1,CT2,CT3,CT_Avg,total_current,state,created_at,therm_temp,vibration
0,0.000000,11.271318,14.648021,8.639779,25.919338,ON,2024-03-26T04:17:37.000000Z,26.383165,1902
1,12.206779,13.903328,10.293859,12.134656,36.403969,ON,2024-03-26T06:07:21.000000Z,31.782297,1903
2,1.157038,1.262079,1.147739,1.188952,3.566856,ON,2024-03-27T01:17:40.000000Z,25.015078,1850
3,1.429621,1.462852,1.459629,1.450701,4.352102,ON,2024-03-27T01:17:45.000000Z,24.980755,1931
4,1.383395,1.581238,1.362355,1.442329,4.326988,ON,2024-03-27T01:17:50.000000Z,25.004904,1815
...,...,...,...,...,...,...,...,...,...
15404,1.190704,1.083099,1.135036,1.136280,3.408839,ON,2024-08-10T06:18:13.000000Z,41.699657,1873
15405,3.768695,3.919246,3.917646,3.868529,11.605587,ON,2024-08-10T06:18:18.000000Z,41.670242,1901
15406,3.933802,3.982554,3.588322,3.834893,11.504679,ON,2024-08-10T06:18:23.000000Z,41.643696,1823
15407,1.279599,1.272777,1.131611,1.227996,3.683987,ON,2024-08-10T06:18:53.000000Z,41.705498,1769


In [26]:
data['created_at'].value_counts()

created_at
2024-03-26T04:17:37.000000Z    1
2024-07-21T06:22:29.000000Z    1
2024-07-21T05:23:20.000000Z    1
2024-07-21T05:23:25.000000Z    1
2024-07-21T05:23:46.000000Z    1
                              ..
2024-05-23T03:21:28.000000Z    1
2024-05-23T03:21:33.000000Z    1
2024-05-23T03:21:38.000000Z    1
2024-05-23T03:21:43.000000Z    1
2024-08-10T06:19:13.000000Z    1
Name: count, Length: 15409, dtype: int64

In [27]:
# Saving the dataset to a new csv file
data.to_csv('./CleanData/data.merged.csv', index=False)